# Продвинутый Python, семинар 10

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Романченко Полина

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

Так как мы проходим веб-разработку, то это явно не та вещь, которая делается через colab. Поэтому здесь написан просто код, который можно воспроизвести локально

## Базы данных в Django

Итак, теперь для того, чтобы сделать что-то разумное, нам необходимо иметь базу даннных, внутри которой мы будем что-либо хранить. Понятное дело, что это все добро нужно постоянно, поэтому Django и здесь позаботилось о том, чтобы можно было все сделать легко и из под коробки.

Для начала идем в settings.py и посмотрим, что там лежит:

```
# Database
# https://docs.djangoproject.com/en/4.1/ref/settings/#databases

DATABASES = {
    "default": {
        "ENGINE": "django.db.backends.sqlite3",
        "NAME": BASE_DIR / "db.sqlite3",
    }
}
```

Что здесь может быть:

* ENGINE - какой диалект SQL используем (PostrgreSQL, MySQL, SQLite3, Oracle)
 
* NAME - Название

* USER - Логин

* PASSWORD - Пароль

* HOST - на каком хосте лежит

Базово (для каких-то локальных вещей) нам подходит простой SQLite3 (он не требует ни указания логина/пароля, единственное, что требудется - указать в Name расположение. Про коннекторы мы поговорим через лекцию)

Что же, на чем строить добро у нас есть, теперь давайте заводить таблички!

Сделаем вначале:

```
python manage.py startapp posts
```

Что делает данная команда? Создает необходимые компоненты для взаимодействия (у нас получилась папка posts, где мы с вами будем что-нибудь хранить).

Первое, что мы хотим сделать - это проиницилизировать нужные таблицы. Идем в models.py и создаем все, что нам нужно. Пойдем по логике, что опять делаем Твиттер:

* Пользователи

* Твиты

* Реплаи (давайте не считать их отдельными постами)

Внутри Django DB строится через модуль ```django.db```, внутри которого есть ```models```

[Документация](https://docs.djangoproject.com/en/4.1/topics/db/models/)

В чем прелесть Models? Базово: мы создаем таблицу без использования SQL (то есть он самостоятельно скомпилирует это в SQL и выполнит за вас), наличие сразу id (самостоятельно указывать не надо, сам все сделает, умница какой), поддерживает каскадность удаления etc (если пользователь удаляется, например, то надо удалить все его посты и все реплаи)

Но давайте с простого и просто поговорим про создание таблиц и полей:

* CharField - строчки

* DataField (DateTimeField) - поле с датой

* FileField, ImageField - поле с файлом/картинкой

* URLField - поле с ссылкой

* IntegerField - число

Давайте для примера сделаем таблицу для юзеров:

In [ ]:
from django.db import models

class User(models.Model):
    name = models.CharField(maxlength=30)
    username = models.CharField(maxlength=30)
    city = models.CharField(maxlength=60)
    country = models.CharField(maxlength=30)
    bio = models.CharField(maxlength=300)
    email = models.EmailField()

### Задание 1.

Составить такие же таблицы для Твитов (Tweets) и ответов (Replies)

*Вот тут надо еще проговорить про models.ForeignKey(class, on_delete=models.CASCADE), а также про default параметр*

Окей, составили что-то, давайте попробуем сделать так, чтобы мы поняли, что оно работает. Идем в настройки и меняем:

```
INSTALLED_APPS = [
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
    "posts"
]
```


А теперь проверим модели:

```
python manage.py makemigrations
```

Ура, все получилось!

## Заводим данные!

Начнем с простого: как сделать insert?

По сути models.Model есть ни что иное, как наш любимый датакласс, а поэтому создать инстанс просто:

```
p = User(name="Timur", username="Palladain",
    city="Moscow", country="Russia", bio="Hah", email="example@example.com")
```

А записать - это простое ```p.save()```, он превратит ваш инстанс в INSERT и сделает все за вас. UPDATE работает также - что-то меняете в словаре, делаете save() и все поменяется

А как удалить? ```p.delete()```. Подытожим:

* Создание строки - как в датаклассе

* Сохранение/изменение - через save()

* Удаление - через delete()

## Делаем select!

Давайте сделаем все и сразу, а потом будем разбираться

```
SELECT count(*)
FROM Tweets
WHERE country="Russia"
```

```
SELECT id
FROM Tweets
WHERE city='Moscow'
ORDER BY id DESC
LIMIT 10
```

Первый запрос:

In [ ]:
from django.db.models import Count

Tweets.objects.filter(country="Russia").aggregate(Count("id"))

Второй запрос:

In [ ]:
Tweets.objects.filter(city="Moscow").order_by("-id")[:10]

А если нам нужен только 1 объект (потому что все запросы выдают список), то можно сделать вот так:

In [ ]:
Tweets.objects.get(username="Palladain") #можно использовать только тогда, когда вы уверены, что такой объект 1, иначе ошибка

И еще чуть-чуть посложнее:

```
select city, count(id)
from Tweets
group by city
```

In [ ]:
Tweets.objects.values('city').annotate(dcount=Count('id'))

## DDL

А вот тут есть небольшая проблема: также нативно сделать ALTER и DROP уже не получится :с (придется уже копаться внутри БД)

## Админка

Одна из крутых фич Django - это наличие админки, с помощью которой можно всем управлять!

Создадим вначале админа:

```
python manage.py createsuperuser

```

Он попросит логин-почту (можно указать любую) и пароль. После того, как мы это все ввели, мы можем ввести все эти credentials, чтобы зайти в админку

Иииии... А где все таблицы, которые мы создали? А для них надо еще зарегистрировать их в админке, для этого идем в admin.py:

```
from django.contrib import admin
from posts.models import User, Tweets, Replies

admin.site.register(User) - зарегистрировали, что admin имеет доступ к User
admin.site.register(Tweets)
admin.site.register(Replies)
```

С админкой можно также играть, менять ее конфигурации, что отображается etc ([что можно сделать](https://django.fun/ru/docs/django/4.1/ref/contrib/admin/))

## Формы

Чего не хватает для того, чтобы вся наша модель в итоге нормально заработала? Правильно, не хватает форм, как же их заполнять?

Для этого внутри Django тоже есть удобные средства)

Какие у нас есть формы?

* Для поиска (которые должны возвращать некоторый GET)

* Для постинга (которые должны возврашать некоторый POST)

Для GET все достаточно просто: мы получаем наш запрос (через requets.get) и далее производим все манипуляции. А что же для POST? Для них есть отдельный класс, которая называется Form

```
from django import forms

class NameForm(forms.Form):
    your_name = forms.CharField(label='Your name', max_length=100)
```

Казалось бы, зачем нам отдельная форма еще? Дело в простоте, ведь теперь можно сделать вот так:

```
def get_name(request):
    if request.method == 'POST':
        form = NameForm(request.POST)
    return render(request, 'name.html', {'form': form})
```

То есть можно взять и запихнуть все это добро сразу же в необходимый результат, который потом можно записать в ту же таблицу etc

Осталось только одно: а как он поймет, где какое поле? Ответ достаточно прост - из разметки HTML, как уже обсуждалось) (через name в качестве аргумента в тэге HTML)

## Что будем в ДЗ?

А в ДЗ вам будет предложено два варианта:

* Собрать это все в общий красивый сайт (либо на flask, либо на django)

* Или же собрать красивые дашборды на Dash или Streamlit (о них на следующей лекции и семинаре)

Как задеплоить добро и референс к ДЗ будет в понедельник, stay tuned :з

## Животное дня

![](https://images.techinsider.ru/upload/img_cache/e8e/e8eb6b3287f8460a27457aac6395a812_cropped_1332x888.jpg)

Сегодня у нас будет самое мелкое животное из представленных - это тихоходка (или водяной медведь). Их более 750 видов, и их можно увидеть под микроскопом (что и увидели ученые еще в XVIII веке). Выглядят очень мило (особенно как они пермещаются)

Но что в них более удивительно, так это их способность выживать в самых экстремальных условиях: давление, космическое излучение (их отправляли в космос и под сильным космическим излучением выжило более 50% тихоходок), температуры, близкие к абсолютному нулю, сухость. Вот кто реально выживет после ядерной войны

Секрет их выносливости - это состояние анабиоза, в которых они впадают. Содержание воды уменьшается до 1%, а метаболизм падает практически до нуля (и они сворачиваются в такой клубочек). Причем в состоянии анабиоза они могут находиться более 100 лет, при этом оживая достаточно быстро. Так что если кто-то и поселится первым на Марсе или на другой планете, то это явно будут тихоходки, а не люди)